In [10]:
import pandas as pd
import numpy as np
from random import choice, choices
import datetime

treatment_df = pd.read_csv(r'Item Code and Treatments/treatment_with_details.csv')

# Input parameters
total_price = 1862464  # Total price to reach as a stopping condition
price_basis = 'Australia'  # Can be 'Indonesia' or 'Australia'
num_unique_patients = 100  # Number of unique patients

# Generate unique patient IDs and their ages
patient_ids = [f'Patient_{str(i).zfill(3)}' for i in range(1, num_unique_patients + 1)]
patient_age_ids = np.random.randint(18, 80, size=num_unique_patients)
patient_age_map = dict(zip(patient_ids, patient_age_ids))

# Define dentist providers with discrete probabilities
dentist_providers = ['RE', 'BEC', 'CQ', 'DA', 'WS']
dentist_probabilities = [0.54, 0.003, 0.27, 0.002, 0.185]

# Random date generator
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 12, 31)

def random_date(start, end):
    return start + datetime.timedelta(days=np.random.randint(0, (end - start).days))

# DataFrame initialization
transactions = []

# Accumulated total price tracker
current_total_price = 0

while current_total_price <= total_price:
    # Randomly select a treatment
    treatment = treatment_df.sample(1).iloc[0]
    
    if price_basis == 'Indonesia':
        price = treatment['total_price_IDR']
        material_cost = treatment['total_cost_IDR']
        duration = treatment['total_duration']
    else:  # 'Australia'
        price = treatment['total_price_AUD']
        material_cost = treatment['total_cost_AUD']
        duration = treatment['total_duration']
    
    # Check if adding this transaction exceeds total_price
    if current_total_price + price > total_price:
        break
    
    # Randomly select a patient and get their age
    patient_id = choice(patient_ids)
    age = patient_age_map[patient_id]
    
    # Randomly select a provider based on the given probabilities
    provider = choices(dentist_providers, dentist_probabilities)[0]
    
    # Create a new transaction
    transaction = {
        'Patient ID': patient_id,
        'Age': age,
        'Treatment': treatment['Treatment'],
        'Duration': duration,
        'Date': random_date(start_date, end_date),
        'Provider Dentist': provider,
        'Price': price,
        'Material Cost': material_cost
    }
    
    # Append the transaction to the list
    transactions.append(transaction)
    
    # Update the accumulated total price
    current_total_price += price

# Create the final DataFrame
patient_transaction_df = pd.DataFrame(transactions)

patient_transaction_df['Date'] = pd.to_datetime(patient_transaction_df['Date'])

# Display the generated DataFrame
patient_transaction_df


,Patient ID,Age,Treatment,Duration,Date,Provider Dentist,Price,Material Cost
0,Patient_028,52,Full banding bracket placement,30.0,2023-09-03,RE,153.0,46
1,Patient_098,45,Caries Xrays,60.0,2023-06-23,RE,24.0,8
2,Patient_080,42,Consultation and TP presentation,45.0,2023-06-24,WS,100.0,30
3,Patient_036,37,Complete Denture (Atas Bawah),170.0,2023-06-05,RE,871.0,261
4,Patient_037,65,RC Minor Package - Back,550.0,2023-05-21,RE,1687.0,507
...,...,...,...,...,...,...,...,...
998,Patient_030,65,Reattachment of fixed wire retainer,120.0,2023-11-20,RE,251.0,75
999,Patient_005,20,Metal partial denture treatment of upper parti...,320.0,2023-08-02,RE,1377.0,414
1000,Patient_065,54,Deep Gum treatment,80.0,2023-01-26,RE,206.0,61
1001,Patient_007,24,Monit 1,120.0,2023-05-21,CQ,209.0,62


In [11]:
dentist_contributions = patient_transaction_df.groupby('Provider Dentist')['Price'].sum()

#change name of Price column to Sales Contribution
dentist_contributions = dentist_contributions.reset_index().rename(columns={'Price': 'Sales Contribution'})
dentist_contributions

,Provider Dentist,Sales Contribution
0,BEC,206.0
1,CQ,523736.0
2,DA,50.0
3,RE,937452.0
4,WS,400335.0


In [13]:
dentist_fee = 20 # per 60 minutes of duration
patient_transaction_df['Dentist Fee'] = patient_transaction_df['Duration'] / 60 * dentist_fee
patient_transaction_df['Net Sales'] = patient_transaction_df['Price'] - patient_transaction_df['Material Cost'] - patient_transaction_df['Dentist Fee']

# take the year and the month from column 'Date' as new columns
patient_transaction_df['Year'] = patient_transaction_df['Date'].dt.year
patient_transaction_df['Month'] = patient_transaction_df['Date'].dt.month

# groupby 'Year' and 'Month' and sum 'Net Sales' 
monthly_net_sales = patient_transaction_df.groupby(['Year', 'Month'])['Net Sales'].sum().reset_index()


# make month column as string of Jan, Feb, etc
monthly_net_sales['Month'] = monthly_net_sales['Month'].apply(lambda x: datetime.date(1900, x, 1).strftime('%B'))

monthly_net_sales


,Year,Month,Net Sales
0,2023,January,118980.000000
1,2023,February,100997.666667
2,2023,March,105166.333333
3,2023,April,70553.000000
4,2023,May,109149.000000
5,2023,June,113605.000000
6,2023,July,92876.333333
7,2023,August,92038.000000
8,2023,September,123700.666667
9,2023,October,102180.666667
